In [ ]:
import base64
import mysql.connector as mysql
import datetime
import pandas as pd
import time

#######
## INIT
#######

# load api key
secret = {}
with open('secret.txt') as f:
    lines = f.readlines()
    for line in lines:
        secret[line.split("=")[0]] = line.split("=")[1].replace("\n","")

# Load blacklist

In [ ]:
blacklist = pd.read_csv('blacklist.csv', header=None, names=['handle'])
blacklist.drop_duplicates(inplace=True)
blacklist.shape

In [ ]:
blacklist = blacklist['handle'].tolist()

# Update blacklist table

## select rows already in blacklist

In [ ]:
db = mysql.connect(host=secret['DBHOST'],user=secret['DBUSER'],passwd=secret['DBPASS'],database=secret['DBTABLE'])
cursor = db.cursor()
query = f'SELECT twitter_username FROM blacklist;'
cursor.execute(query)
records_rewards = cursor.fetchall()
cursor.close()
db.close()
# convert to dataframe
blacklist_db = []
for record in records_rewards:
    blacklist_db.append(record[0])

## write to blacklist table

In [ ]:
for twitter_username in blacklist:
    if twitter_username not in blacklist_db:
        # write to blacklist table 
        db = mysql.connect(host=secret['DBHOST'],user=secret['DBUSER'],passwd=secret['DBPASS'],database=secret['DBTABLE'])
        cursor = db.cursor()
        query = "INSERT INTO blacklist (twitter_username) VALUES (%s);"
        values = (twitter_username,)
        cursor.execute(query, values)
        db.commit()
        cursor.close()
        db.close()  

# Remove blacklist users from twitter and rewards tables

In [ ]:
for i, handle in enumerate(blacklist):
    
    print(f"Removing blacklist user {i+1}/{len(blacklist)} {handle}")

    # delete from twitter
    db = mysql.connect(host=secret['DBHOST'],user=secret['DBUSER'],passwd=secret['DBPASS'],database=secret['DBTABLE'])
    cursor = db.cursor()
    query = "DELETE FROM twitter WHERE twitter_handle = %s;"
    values = (handle,)
    cursor.execute(query, values)
    db.commit()
    print(cursor.rowcount, "record updated")
    cursor.close()
    db.close()  

    # delete from rewards
    db = mysql.connect(host=secret['DBHOST'],user=secret['DBUSER'],passwd=secret['DBPASS'],database=secret['DBTABLE'])
    cursor = db.cursor()
    query = "DELETE FROM rewards WHERE twitter_handle = %s;"
    values = (handle,)
    cursor.execute(query, values)
    db.commit()
    print(cursor.rowcount, "record updated")
    cursor.close()
    db.close()  
    